# Install libraries and dependencies (it will take 5-10 minutes)

In [ ]:
%%bash -x

# ============================================
# 0. CLEAN EVERYTHING
# ============================================
rm -rf /content/py310
rm -rf /content/ProbPose
rm -rf /content/chumpy-fixed

# ============================================
# 1. Install Python 3.10
# ============================================
add-apt-repository ppa:deadsnakes/ppa -y
apt update -y
apt install -y python3.10 python3.10-dev python3.10-distutils python3.10-venv
curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# ============================================
# 2. Create venv
# ============================================
python3.10 -m venv /content/py310
/content/py310/bin/python -m pip install --upgrade pip setuptools wheel

# ============================================
# 3. INSTALL NUMPY 1.26 FIRST (LOCK IT)
# ============================================
/content/py310/bin/python -m pip install numpy==1.26.4
echo "numpy<2" > /content/py310/constraints.txt

# ============================================
# 4. Install PyTorch 2.0.1 + cu118
# ============================================
/content/py310/bin/python -m pip install torch==2.0.1+cu118 \
    torchvision==0.15.2+cu118 \
    --extra-index-url https://download.pytorch.org/whl/cu118 \
    -c /content/py310/constraints.txt

# ============================================
# 5. Install MMCV + mmengine + mmdet + mmpretrain
# ============================================
/content/py310/bin/python -m pip install -U openmim -c /content/py310/constraints.txt
/content/py310/bin/mim install mmengine -c /content/py310/constraints.txt
/content/py310/bin/mim install "mmcv==2.1.0" -c /content/py310/constraints.txt
/content/py310/bin/mim install "mmdet==3.3.0" -c /content/py310/constraints.txt
/content/py310/bin/mim install "mmpretrain==1.2.0" -c /content/py310/constraints.txt

# ============================================
# 6. Local chumpy (works with NumPy 1.26)
# ============================================
mkdir -p /content/chumpy-fixed/chumpy

cat << 'EOF' > /content/chumpy-fixed/chumpy/__init__.py
from .ch import Ch
from .utils import *
EOF

cat << 'EOF' > /content/chumpy-fixed/chumpy/ch.py
import numpy as np

class Ch(np.ndarray):
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def compute_r(self):
        return np.asarray(self)

    def compute_dr_wrt(self, wrt):
        return None
EOF

cat << 'EOF' > /content/chumpy-fixed/chumpy/utils.py
import numpy as np

def row(x):
    return np.asarray(x).reshape(1, -1)

def col(x):
    return np.asarray(x).reshape(-1, 1)
EOF

cat << 'EOF' > /content/chumpy-fixed/setup.py
from setuptools import setup, find_packages
setup(
    name='chumpy',
    version='0.1.0',
    packages=find_packages(),
    install_requires=['numpy<2']
)
EOF

/content/py310/bin/python -m pip install /content/chumpy-fixed -c /content/py310/constraints.txt

# ============================================
# 7. Clone ProbPose
# ============================================
git clone https://github.com/sachin-detrax/ProbPose_code.git /content/ProbPose

# ============================================
# 8. Install ProbPose deps
# ============================================
cd /content/ProbPose
/content/py310/bin/python -m pip install -r requirements.txt --no-deps -c /content/py310/constraints.txt
/content/py310/bin/python -m pip install -e . -c /content/py310/constraints.txt

# ============================================
# 9. Download weights
# ============================================
cd /content
wget -O ProbPose-s.pth "https://huggingface.co/vrg-prague/ProbPose-s/resolve/main/ProbPose-s.pth"

echo "==============================================="
echo "ProbPose setup COMPLETED with NumPy LOCKED."
echo "==============================================="


# Run Demo

In [ ]:
!MPLBACKEND=Agg /content/py310/bin/python /content/ProbPose/demo/image_demo.py \
    /content/ProbPose/demo/resources/CropCOCO_single_example.jpg \
    /content/ProbPose/configs/body_2d_keypoint/topdown_probmap/coco/td-pm_ProbPose-small_8xb64-210e_coco-256x192.py \
    /content/ProbPose-s.pth \
    --out-file /content/result.jpg \
    --draw-heatmap
